In [21]:
import random
import cassiopeia as cass
from cassiopeia import Queue, Patch
from cassiopeia import Summoner, MatchHistory, Match
from cassiopeia.data import Continent
import arrow

### 1: Start

In [22]:
cass.set_riot_api_key("RGAPI-755a0f0f-c83a-425d-a1bb-f66157bd1c7e")
region = "EUW"

### 2: Pobranie 300 najlepszych graczy z challengera

In [23]:
best_300_challenger = cass.get_challenger_league(queue=cass.Queue.ranked_solo_fives, region=region)
best = best_300_challenger[0]
print(best.summoner)

Summoner(id=_Af3HNv4O525zDKyP6XibUjf7snAp7zytc6L2kgD14VZBPw-, account_id=gleL3tvseKqQReQdXZt7FO0iknF-Ctg0xez_i93TN5cBjKqYQ9YEWPbc, name='Fatihcan', puuid='BgSWeXqw8HOns1pJ-PJGf-kCDhLvE7fzuYfwgE9QrQhNO6iwgdUkAjr-XuDk40LyPa2MoKNUdH8eIA')


### 3: Pobranie 300 summonerów

In [24]:
summoners = []
for best in best_300_challenger:
    summoner = Summoner(name=best.summoner.name, region=region)
    summoners.append(summoner)

In [25]:
print(summoners[0].id, summoners[0].name)

_Af3HNv4O525zDKyP6XibUjf7snAp7zytc6L2kgD14VZBPw- Fatihcan


### 4: Pobranie id championów

Unfortunately since we are iterating over the match history and accessing the summoner's champion for each match,
we need to know what version of the static data the champion should have. To avoid pulling many different
static data versions, we will instead create a {champion_id -> champion_name} mapping and just access the champion's
ID from the match data (which it provides directly).

In [26]:
champion_id_to_name_mapping = {
    champion.id: champion.name for champion in cass.get_champions(region=region)
}

### 5: Pobieranie historii meczy

Zbierane są dane od patcha ??11.1?? do najnowszego.

In [27]:
# patch = Patch.from_str("3.1", region=region)
# print(patch.start)

#
# def filter_match_history(summoner, patch):
#     end_time = patch.end
#     if end_time is None:
#         end_time = arrow.now()
#     match_history = MatchHistory(
#         puuid=summoner.puuid,
#         queue=Queue.ranked_solo_fives,
#         start_time=patch.start,
#         end_time=end_time,
#         continent=Continent.europe
#     )
#     print('len match_history:', len(match_history))
#     return match_history



In [28]:
def get_matches_history(summoners):
    matches_history = []
    for summoner in summoners:
        matches_history += summoner.match_history
    return matches_history


matches_history = get_matches_history([summoners[0]])

Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/BgSWeXqw8HOns1pJ-PJGf-kCDhLvE7fzuYfwgE9QrQhNO6iwgdUkAjr-XuDk40LyPa2MoKNUdH8eIA/ids?start=0&count=100
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/BgSWeXqw8HOns1pJ-PJGf-kCDhLvE7fzuYfwgE9QrQhNO6iwgdUkAjr-XuDk40LyPa2MoKNUdH8eIA/ids?start=100&count=100
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/BgSWeXqw8HOns1pJ-PJGf-kCDhLvE7fzuYfwgE9QrQhNO6iwgdUkAjr-XuDk40LyPa2MoKNUdH8eIA/ids?start=200&count=100
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/BgSWeXqw8HOns1pJ-PJGf-kCDhLvE7fzuYfwgE9QrQhNO6iwgdUkAjr-XuDk40LyPa2MoKNUdH8eIA/ids?start=300&count=100
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/BgSWeXqw8HOns1pJ-PJGf-kCDhLvE7fzuYfwgE9QrQhNO6iwgdUkAjr-XuDk40LyPa2MoKNUdH8eIA/ids?start=400&count=100
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/BgSWeXqw8HOns1pJ-PJGf-kCDhLvE7f

In [31]:
from parameters import match_features_names


def get_match_participants_features(match):
    participants_features = {}
    for i, participant in enumerate(match.participants):
        key = "User" + str(i)
        participants_features[key] = {feature_name: participant.stats.__getattribute__(feature_name) for feature_name in
                                      match_features_names}
    return participants_features


for match in matches_history[0:1]:
    print(match.to_json())
    # match_features = get_match_participants_features(match)
    # print(match_features)

{"privateGame": false, "creation": "2022-12-14T15:50:26.207000+00:00", "tournamentCode": "", "matchId": 6189110915, "version": "12.23.483.5208", "mode": "CLASSIC", "duration": 1113, "teams": [{"bans": [{"championId": 517, "pickTurn": 1}, {"championId": -1, "pickTurn": 2}, {"championId": 200, "pickTurn": 3}, {"championId": 77, "pickTurn": 4}, {"championId": 114, "pickTurn": 5}], "participants": [{"getBackPings": 4, "enemyMissingPings": 9, "baitPings": 0, "endedInEarlySurrender": false, "puuid": "1YFetO1Ex8zhAgIRTtKZ6ufU1KRhZEmog6pcvq56g_LLljxSqo6tEhe2_EJbdJZKp2uJS1zeKM1omw", "summonerId": "LcdlMBpSl5Ek6LxnaGXH3zRLIJN013FCpOapYG6vaRxuCvY", "summonerSpellFId": 12, "needVisionPings": 0, "dangerPings": 0, "riotIdTagline": "", "enemyVisionPings": 0, "commandPings": 7, "participantId": 1, "platformId": "EUW1", "stat_perks": {"defense": 5002, "flex": 5008, "offense": 5005}, "holdPings": 0, "stats": {"lane": "NONE", "killingSprees": 0, "inhibitorsLost": 1, "baronKills": 0, "trueDamageDealt": 0,